In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import numpy as np
import torch
import torch.nn as nn
import torch.nn.init as init
import torch.nn.functional as F

In [ ]:
prediction_hourly = np.load('/content/drive/MyDrive/COMP9491_ASTGCN_Model/Dataset_PEMS07/Best_Epoch_Saved_Recent/recentk4output_epoch_17_test.npz')
prediction_daily = np.load('/content/drive/MyDrive/COMP9491_ASTGCN_Model/Dataset_PEMS07/Best_Epoch_Saved_Recent/Daily_K4_output_epoch_18_test.npz')

##### Printing the Actual Target Tensor Shape

In [ ]:
target_tensor = prediction_hourly['data_target_tensor']
print(target_tensor.shape)

(5641, 883, 12)


In [ ]:
print(prediction_hourly['prediction'].shape)
print(prediction_daily['prediction'].shape)

(5641, 883, 12)
(5528, 883, 12)


In [ ]:
prediction_hourly['prediction'][0]

array([[475.33215, 485.2931 , 495.83673, ..., 532.2502 , 538.00946,
        542.5406 ],
       [364.97992, 361.56674, 359.54962, ..., 344.89423, 342.54596,
        339.56497],
       [568.4268 , 576.91   , 586.3029 , ..., 621.3616 , 626.02185,
        629.18414],
       ...,
       [573.9524 , 572.8489 , 571.11975, ..., 559.68243, 556.94696,
        556.97424],
       [225.66855, 226.14908, 226.95058, ..., 219.43892, 218.52197,
        217.20898],
       [564.71967, 564.3108 , 564.25745, ..., 553.9462 , 552.39624,
        549.7503 ]], dtype=float32)

In [ ]:
prediction_hourly['prediction'][0].shape

(883, 12)

In [ ]:
print(type(prediction_hourly['prediction']))

<class 'numpy.ndarray'>


In [ ]:
prediction_hourly_np = prediction_hourly['prediction']
#prediction_hourly_target = prediction_hourly['data_target_tensor']

In [ ]:
# Create the original and new indices
original_indices = np.arange(prediction_hourly_np.shape[0])  # [0, 1, 2, ..., 5640]
new_indices = np.linspace(0, prediction_hourly_np.shape[0] - 1, num=5528)

In [ ]:
import numpy as np
import torch
from scipy.interpolate import interp1d

In [ ]:
interpolated_prediction = np.zeros((5528, 883, 12))

In [ ]:
for i in range(prediction_hourly_np.shape[1]):  # Loop over the second dimension (883)
    for j in range(prediction_hourly_np.shape[2]):  # Loop over the third dimension (12)
        # Create the interpolation function for the 1D array prediction_hourly_np[:, i, j]
        f = interp1d(original_indices, prediction_hourly_np[:, i, j], kind='linear')

        # Interpolate values at new_indices and assign to the new tensor
        interpolated_prediction[:, i, j] = f(new_indices)


In [ ]:
interpolated_prediction.shape

(5528, 883, 12)

In [ ]:
prediction_daily['prediction'].shape

(5528, 883, 12)

In [ ]:
interpolated_prediction[0]

array([[475.33215332, 485.29309082, 495.83673096, ..., 532.25018311,
        538.00946045, 542.54058838],
       [364.97991943, 361.56674194, 359.54962158, ..., 344.89422607,
        342.54595947, 339.56497192],
       [568.42681885, 576.90997314, 586.30291748, ..., 621.36157227,
        626.02185059, 629.18414307],
       ...,
       [573.95239258, 572.84887695, 571.11975098, ..., 559.68243408,
        556.94696045, 556.97424316],
       [225.66854858, 226.14907837, 226.95057678, ..., 219.43891907,
        218.52197266, 217.20898438],
       [564.71966553, 564.31079102, 564.25744629, ..., 553.94622803,
        552.39624023, 549.75030518]])

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F

In [ ]:
class Temporal_Attention_layer(nn.Module):
    def __init__(self, in_channels, num_of_vertices, num_of_timesteps):
        super(Temporal_Attention_layer, self).__init__()
        self.U1 = nn.Parameter(torch.FloatTensor(num_of_vertices))
        self.U2 = nn.Parameter(torch.FloatTensor(in_channels, num_of_vertices))
        self.U3 = nn.Parameter(torch.FloatTensor(in_channels))
        self.be = nn.Parameter(torch.FloatTensor(1, num_of_timesteps, num_of_timesteps))
        self.Ve = nn.Parameter(torch.FloatTensor(num_of_timesteps, num_of_timesteps))

    def forward(self, x):
        '''
        :param x: (batch_size, N, F_in, T)
        :return: (B, T, T)
        '''
        _, num_of_vertices, num_of_features, num_of_timesteps = x.shape

        lhs = torch.matmul(torch.matmul(x.permute(0, 3, 2, 1), self.U1), self.U2)
        # x:(B, N, F_in, T) -> (B, T, F_in, N)
        # (B, T, F_in, N)(N) -> (B,T,F_in)
        # (B,T,F_in)(F_in,N)->(B,T,N)

        rhs = torch.matmul(self.U3, x)  # (F)(B,N,F,T)->(B, N, T)

        product = torch.matmul(lhs, rhs)  # (B,T,N)(B,N,T)->(B,T,T)

        E = torch.matmul(self.Ve, torch.sigmoid(product + self.be))  # (B, T, T)

        E_normalized = F.softmax(E, dim=1)
        ### Temporal Attention score measures the importance of each timesteps fore a node in the Graph
        return E_normalized

In [ ]:
import torch
import torch.nn as nn
import torch.nn.init as init

class MultiComponentFusion(nn.Module):
    def __init__(self, num_nodes, num_timesteps,in_channels, hourly_bias):
        super(MultiComponentFusion, self).__init__()

        # Initialize weight matrices for each component
        self.Wh = nn.Parameter(torch.empty(num_nodes, num_timesteps))
        self.Wd = nn.Parameter(torch.empty(num_nodes, num_timesteps))

        # Apply Xavier uniform initialization
        init.xavier_uniform_(self.Wh)
        init.xavier_uniform_(self.Wd)

        # Apply sigmoid to ensure weights are between 0 and 1
        self.sigmoid = nn.Sigmoid()
        #self.TAt = Temporal_Attention_layer(in_channels, num_nodes, num_timesteps)

        self.hourly_bias = hourly_bias

    def forward(self, y_actual, y_predicted_hourly, y_predicted_daily):
        # Convert NumPy arrays to PyTorch tensors
        y_actual = torch.tensor(y_actual).float() # Convert to tensor and cast to float
        y_predicted_hourly = torch.tensor(y_predicted_hourly).float() # Convert to tensor and cast to float
        y_predicted_daily = torch.tensor(y_predicted_daily).float() # Convert to tensor and cast to float

        # Apply sigmoid to weights
        Wh = self.sigmoid(self.Wh).detach()
        Wd = self.sigmoid(self.Wd).detach()

        # Expand weights to match input dimensions
        Wh = Wh.unsqueeze(0)  # Shape becomes (1, 883, 12)
        Wd = Wd.unsqueeze(0)

        # Perform element-wise multiplication and sum
        #Y_fused = Wh * y_predicted_hourly + Wd * y_predicted_daily
        # Cast input tensors to float32

        #Wh = Wh * self.hourly_bias
        #Wd = Wd * (1 - self.hourly_bias)




        # Perform element-wise multiplication
        Yh_weighted = Wh * y_predicted_hourly
        Yd_weighted = Wd * y_predicted_daily


        Y_fused = Yh_weighted + Yd_weighted


        #num_of_samples, num_of_nodes, num_of_timesteps = Y_fused.shape
        #Y_fused = Y_fused.unsqueeze(2)
        #print(Y_fused.shape)
        #temporal_Att = self.TAt(Y_fused)
        #Y_fused = Y_fused.squeeze(2)
        #Y_fused = torch.matmul(Y_fused, temporal_Att).reshape(self.num_of_samples, num_nodes, num_timesteps)

        return Y_fused


num_samples_test_set = 5528
num_nodes = 883
num_timesteps = 12
in_channels = 1
num_of_vertices = num_nodes
hourly_bias = 0.7

# Create an instance of the fusion model
##USE_CUDA = torch.cuda.is_available()
#DEVICE = torch.device('cuda:0')
#print("CUDA:", USE_CUDA, DEVICE)
fusion_model = MultiComponentFusion(num_nodes, num_timesteps, in_channels, hourly_bias)

# Example
Y_actual = prediction_daily['data_target_tensor']
Yh = interpolated_prediction
Yd = prediction_daily['prediction']

Y_final = fusion_model(Y_actual, Yh, Yd)

In [ ]:
Y_final.shape

torch.Size([5528, 883, 12])

In [ ]:
Y_final[0]

tensor([[499.6994, 495.6994, 511.5723,  ..., 512.6046, 505.3198, 518.7771],
        [323.1136, 325.2087, 323.0311,  ..., 326.2578, 319.0840, 320.9930],
        [584.2102, 585.0057, 592.1371,  ..., 599.8114, 598.7515, 616.8837],
        ...,
        [539.7239, 538.2589, 517.8902,  ..., 519.0676, 520.3436, 521.6851],
        [236.8844, 239.9137, 235.7339,  ..., 228.9604, 221.4013, 224.3575],
        [566.1902, 577.1260, 565.9804,  ..., 557.3352, 560.6555, 564.1551]])

In [ ]:
import torch.optim as optim

In [ ]:
from sklearn.metrics import mean_absolute_error

# Ensure tensors are detached and moved to CPU before converting to numpy
Y_final_np = Y_final.detach().cpu().numpy()
Y_actual_np = Y_actual

# Flatten the tensors to 1D arrays for MAE calculation
Y_final_flat = Y_final_np.reshape(-1, 1)
Y_actual_flat = Y_actual_np.reshape(-1, 1)

# Calculate Mean Absolute Error
mae = mean_absolute_error(Y_final_flat, Y_actual_flat)

print("Mean Absolute Error:", mae)


Mean Absolute Error: 79.35147


In [ ]:
from sklearn.metrics import mean_squared_error
import numpy as np

# Ensure tensors are detached and moved to CPU before converting to numpy
Y_final_np = Y_final.detach().cpu().numpy()
Y_actual_np = Y_actual

# Flatten the tensors to 1D arrays for RMSE calculation
Y_final_flat = Y_final_np.reshape(-1, 1)
Y_actual_flat = Y_actual_np.reshape(-1, 1)

# Calculate Mean Squared Error
mse = mean_squared_error(Y_final_flat, Y_actual_flat)

# Calculate Root Mean Squared Error
rmse = np.sqrt(mse)

print("Root Mean Squared Error:", rmse)


Root Mean Squared Error: 108.50808
